JSON dataset

In [1]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pandas as pd
#spacy
import spacy
from nltk.corpus import stopwords

#visualization
import pyLDAvis
import pyLDAvis.gensim
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
from bertopic import BERTopic
import re
stopwords = stopwords.words("english")

In [2]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [3]:
# for item in data:
#     if isinstance(item["Description"], str):
#         original_text = item["Description"]
#         # Remove unwanted tags and patterns
#         cleaned_text = remove_tags(original_text)
#         # Apply lemmatization on the cleaned text
#         lemmatized_text = lemmatization(cleaned_text)
#         # Tokenize the lemmatized text
#         tokenized_words = gen_words(lemmatized_text)
#         # Update "Description" with the joined tokenized words
#         item["Description_new"] = " ".join(tokenized_words)
# write_data("data_processed.json", data)
# data = load_data("data_processed.json")
# for item in data:
#     Loop over possible comment fields
#     for i in range(20):  # From Comment to Comment.19
#         comment_key = "Comment" if i == 0 else f"Comment.{i}"
#         if isinstance(item.get(comment_key), str):
#             comment_text = item[comment_key]
#             cleaned_comment = remove_tags(comment_text)
#             lemmatized_comment = lemmatization(cleaned_comment)
#             tokenized_comment = gen_words(lemmatized_comment)
#             item[comment_key] = " ".join(tokenized_comment)
# write_data("data_processed.json", data)
#Extract values and collect unique ones using a set
# unique_categories = set()

# for item in data:
#     if "Custom field (Request Category)" in item:
#         unique_categories.add(item["Custom field (Request Category)"])

# # Print the unique values
# for value in unique_categories:
#     print(value)


# Extract values and collect unique ones using a set
# unique_categories = set()

# for item in data:
#     if "Custom field (Cause of issue)" in item:
#         unique_categories.add(item["Custom field (Cause of issue)"])

# # Print the unique values
# for value in unique_categories:
#     print(value)


In [30]:
# Helper functions
# preprocessing for LDA
def lemmatization(text, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    doc = nlp(text)
    new_text = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
    return " ".join(new_text)

def gen_words(text):
    return gensim.utils.simple_preprocess(text, deacc=True)
import re

def remove_tags(text):
    special_chars = {'{', '[', '(', '*', '!', '/', '"', '\\', ':', ';', '<'}
    # Split the text into words using whitespace
    words = text.split()
    # Create a new list to hold words that do not contain any special characters or digits
    cleaned_words = []
    # Iterate over each word in the list
    for word in words:
        # Check if the word contains any of the special characters or any digits
        if not any(char in word for char in special_chars) and not re.search(r'\d', word):
            cleaned_words.append(word)
    # Rejoin the cleaned words into a string
    return ' '.join(cleaned_words)
  
    

# preprocessing for bertopic


def clean_for_bertopic(text):
    if not isinstance(text, str):
        return ""

    # Remove email headers/footers and metadata
    text = re.sub(r'On .* wrote:', '', text)
    text = re.sub(r'(?i)Yvonne Brown.*?(Director)?', '', text)
    text = re.sub(r'\*P\.*.*?\d{3}-\d{3}-\d{4}', '', text)
    text = re.sub(r'Capital Area Food Bank.*', '', text)
    text = re.sub(r'\|', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+', '', text)

    # Remove all digits
    text = re.sub(r'\d+', '', text)

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize and filter
    words = text.split()
    words = [
        word.lower() for word in words
        if word.isalpha() and len(word) >= 3
    ]

    return ' '.join(words)

def combine_description_and_comments(item):
    base_text = item.get("Description", "")
    comments = []
    
    # Include all comment fields
    for i in range(20):  # Assuming Comment to Comment.19
        key = "Comment" if i == 0 else f"Comment.{i}"
        comment = item.get(key)
        if isinstance(comment, str):
            comments.append(comment)

    full_text = base_text + " " + " ".join(comments)
    return full_text
def get_top_words(topic_id, model, top_n=10):
    topic = model.get_topic(topic_id)
    if topic is None:
        return ""
    return ", ".join([f"{word}:{weight:.3f}" for word, weight in topic[:top_n]])

def save_bertopic_to_csv(model, docs, topics, filename="bertopic_summary.csv", top_n_words=10):
    # Topic summary info
    df_info = model.get_topic_info()

    # Add top words with weights
    df_info["Top_Words"] = df_info["Topic"].apply(
        lambda x: get_top_words(int(x), model) if x != -1 else ""
    )

    # Add example document per topic
    df_docs = pd.DataFrame({"Document": docs, "Topic": topics})
    sample_docs = df_docs.groupby("Topic")["Document"].first().reset_index()
    df_info = df_info.merge(sample_docs, how="left", on="Topic")

    # Save to CSV
    df_info.to_csv(filename, index=False)
    print(f"Saved BERTopic summary to {filename}")
    from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def filter_low_tfidf_words(docs, threshold_percentile=30):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(docs)
    tfidf_scores = X.toarray()
    feature_names = np.array(vectorizer.get_feature_names_out())

    filtered_docs = []
    for i, doc_scores in enumerate(tfidf_scores):
        threshold = np.percentile(doc_scores[doc_scores > 0], threshold_percentile)  # avoid zeros
        keep_indices = np.where(doc_scores >= threshold)[0]
        selected_words = feature_names[keep_indices]
        doc_words = docs[i].split()
        doc_filtered = [word for word in doc_words if word in selected_words]
        filtered_docs.append(" ".join(doc_filtered))

    return filtered_docs
def filter_low_tfidf_words_justdescripton(docs, threshold_percentile=30):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(docs)
    tfidf_scores = X.toarray()
    feature_names = np.array(vectorizer.get_feature_names_out())

    filtered_docs = []

    for i, doc_scores in enumerate(tfidf_scores):
        non_zero_scores = doc_scores[doc_scores > 0]

        if len(non_zero_scores) == 0:
            # If no non-zero TF-IDF, keep original document as-is
            filtered_docs.append(docs[i])
            continue

        threshold = np.percentile(non_zero_scores, threshold_percentile)
        keep_indices = np.where(doc_scores >= threshold)[0]
        selected_words = set(feature_names[keep_indices])

        doc_words = docs[i].split()
        doc_filtered = [word for word in doc_words if word in selected_words]
        filtered_docs.append(" ".join(doc_filtered))

    return filtered_docs

In [31]:
# dataset prepared for bertopic
data_bertopic = load_data("data_case2-1.json")
for ticket in data_bertopic:
    # Clean Description
    if "Description" in ticket and isinstance(ticket["Description"], str):
        ticket["Description"] = remove_tags(ticket["Description"])

    # Clean all comment fields: Comment, Comment.1, ..., Comment.19
    for i in range(20):
        key = "Comment" if i == 0 else f"Comment.{i}"
        if key in ticket and isinstance(ticket[key], str):
            ticket[key] = remove_tags(ticket[key])
write_data("data_bertopic.json", data_bertopic)

Bertopic

In [25]:
# Hyperparameters
embedding_model_name = "all-MiniLM-L6-v2"
min_topic_size = 3

Using only description

In [54]:
data = load_data("data_bertopic.json")
data_all_desc = [item["Description"] for item in data if isinstance(item.get("Description"), str)]

# docs_descriptions = filter_low_tfidf_words_justdescripton(data_all_desc)
topic_model_all_descriptions = BERTopic(min_topic_size=min_topic_size, embedding_model=embedding_model_name)
topics_all_descriptions, probs_all_descriptions = topic_model_all_descriptions.fit_transform(data_all_desc)
save_bertopic_to_csv(
    model=topic_model_all_descriptions,
    docs=data_all_desc,
    topics=topics_all_descriptions,
    filename=f"bertopic_onlydescriptions_processeddata_min_topic_{min_topic_size}.csv"
)

Saved BERTopic summary to bertopic_onlydescriptions_processeddata_min_topic_3.csv


Use Llama2 to extract meaningful topics

In [ ]:
%%bash
brew install cmake pkg-config openblas



In [ ]:
%%bash
cd llama.cpp
make LLAMA_METAL=1


In [1]:
from huggingface_hub import login
login(token="HF_TOKEN_REMOVED")



/Users/alifehmiyildiz/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import pandas as pd
from llama_cpp import Llama
import re

def parse_top_words(text):
    if not isinstance(text, str):
        return []
    return [match.split(':')[0].strip() for match in text.split(',') if ':' in match]

# Load your BERTopic CSV data
df = pd.read_csv("bertopic_onlydescriptions_processeddata_min_topic_3.csv")  # Adjust path if needed

# Initialize llama.cpp
llm = Llama(model_path="/Users/alifehmiyildiz/Desktop/GitHub/FoodTicketAI/llama.cpp/build/tinyllama.gguf", n_ctx=2048)

def generate_topic_label(keywords, docs, max_tokens=32):
    prompt = f"""Return only a short 1 to 3 word topic label.

Keywords:
{', '.join(keywords)}

Examples:
- {docs[0]}
- {docs[1] if len(docs) > 1 else ''}

Topic Label:"""

    response = llm(prompt, max_tokens=max_tokens)
    output = response['choices'][0]['text'].strip()

    # Only take the first line or sentence before a newline, period, or colon
    clean_label = re.split(r"[\n\.\:]", output)[0].strip()

    return clean_label

# List to store results
topic_id_label_pairs = []

for i, row in df.iterrows():
    keywords = parse_top_words(row['Top_Words'])

    try:
        docs = eval(row['Representative_Docs']) if isinstance(row['Representative_Docs'], str) else []
    except:
        docs = []

    if len(keywords) == 0 or len(docs) == 0:
        label = "[insufficient data]"
    else:
        label = generate_topic_label(keywords, docs)

    print(f"Topic {row['Topic']} → {label}")
    topic_id_label_pairs.append({'Topic': row['Topic'], 'Generated_Label': label})

# Convert to DataFrame for display/export
output_df = pd.DataFrame(topic_id_label_pairs)

# Optional: Save to CSV if needed
output_df.to_csv("topic_id_label_only.csv", index=False)

# Optional: Display just the result
print("\nGenerated Topic Labels:")
print(output_df)


llama_model_load_from_file_impl: using device Metal (Apple M1) - 10916 MiB free
llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from /Users/alifehmiyildiz/Desktop/GitHub/FoodTicketAI/llama.cpp/build/tinyllama.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: -

Topic -1 → [insufficient data]


llama_perf_context_print:        load time =     772.28 ms
llama_perf_context_print: prompt eval time =     772.15 ms /    98 tokens (    7.88 ms per token,   126.92 tokens per second)
llama_perf_context_print:        eval time =     419.35 ms /    31 runs   (   13.53 ms per token,    73.92 tokens per second)
llama_perf_context_print:       total time =    1195.10 ms /   129 tokens
Llama.generate: 20 prefix-match hit, remaining 66 prompt tokens to eval


Topic 0 → Delivery


llama_perf_context_print:        load time =     772.28 ms
llama_perf_context_print: prompt eval time =     585.98 ms /    66 tokens (    8.88 ms per token,   112.63 tokens per second)
llama_perf_context_print:        eval time =     425.62 ms /    31 runs   (   13.73 ms per token,    72.83 tokens per second)
llama_perf_context_print:       total time =    1014.99 ms /    97 tokens
Llama.generate: 20 prefix-match hit, remaining 142 prompt tokens to eval


...

Topic 62 → ccda, communication, intentional, information, legal, legally, privileged, st, unlawful, uei, vaccination,


llama_perf_context_print:        load time =     772.28 ms
llama_perf_context_print: prompt eval time =     879.81 ms /   186 tokens (    4.73 ms per token,   211.41 tokens per second)
llama_perf_context_print:        eval time =     460.86 ms /    31 runs   (   14.87 ms per token,    67.27 tokens per second)
llama_perf_context_print:       total time =    1345.14 ms /   217 tokens
Llama.generate: 20 prefix-match hit, remaining 297 prompt tokens to eval


Topic 63 → Request a Change in Delivery Date/Time


llama_perf_context_print:        load time =     772.28 ms
llama_perf_context_print: prompt eval time =    1257.21 ms /   297 tokens (    4.23 ms per token,   236.24 tokens per second)
llama_perf_context_print:        eval time =     464.28 ms /    31 runs   (   14.98 ms per token,    66.77 tokens per second)
llama_perf_context_print:       total time =    1726.32 ms /   328 tokens


Topic 64 → - Quality of produce inspection, quality of produce, quality of produce inspection, quality of produce inspections, quality of produce inspections

Generated Topic Labels:
    Topic                                    Generated_Label
0      -1                                [insufficient data]
1       0                                           Delivery
2       1               Order Produce Pound Limit Last Sales
3       2  - Add (1), Pallet (2), Potatoes (3), Add (4), ...
4       3                                          "Greeing"
..    ...                                                ...
61     60  Food CoDing GuideLines Physical Copies | Sourc...
62     61  - Crisped Ricer, Cornflakes, Oatmeal, Oz, Milk...
63     62  ccda, communication, intentional, information,...
64     63             Request a Change in Delivery Date/Time
65     64  - Quality of produce inspection, quality of pr...

[66 rows x 2 columns]


In [ ]:
# train separate model for each category
from bertopic import BERTopic
from umap import UMAP
import os

def train_bertopic_per_request_category(
    json_path,
    min_topic_size,
    embedding_model_name="all-MiniLM-L6-v2",
    output_dir="bertopic_outputs_per_category"
):
    # Load & group data
    data = load_data(json_path)
    grouped_data = {}
    for item in data:
        category = item.get("Custom field (Request Category)")
        description = item.get("Description")
        if isinstance(category, str) and isinstance(description, str):
            grouped_data.setdefault(category, []).append(description)

    os.makedirs(output_dir, exist_ok=True)

    for category, descriptions in grouped_data.items():
        print(f"\n📂 Processing category: '{category}' ({len(descriptions)} items)")

        # Step 1: Clean
        docs_cleaned = [clean_for_bertopic(desc) for desc in descriptions]
        docs_cleaned = [remove_tags(doc) for doc in docs_cleaned]
        docs_filtered = filter_low_tfidf_words_justdescripton(docs_cleaned)

        if len(docs_filtered) < 10:
            print(f"⚠️  Skipped '{category}' — Not enough data after filtering ({len(docs_filtered)} docs)")
            continue

        # Step 2: Custom UMAP for small batches
        custom_umap = UMAP(
            n_neighbors=min(10, len(docs_filtered) - 1),
            n_components=5,
            metric='cosine'
        )

        # Step 3: Train model
        topic_model = BERTopic(
            min_topic_size=min_topic_size,
            embedding_model=embedding_model_name,
            umap_model=custom_umap
        )
        topics, _ = topic_model.fit_transform(docs_filtered)

        # Step 4: Save to CSV
        safe_category = category.replace("/", "_").replace(" ", "_").lower()
        save_path = os.path.join(output_dir, f"bertopic_{safe_category}.csv")
        save_bertopic_to_csv(
            model=topic_model,
            docs=docs_filtered,
            topics=topics,
            filename=save_path
        )
        print(f"✅ Saved results for '{category}' to: {save_path}")
train_bertopic_per_request_category(
    "data_processed.json",
    min_topic_size,
)

Using comment section and description 

In [57]:
data = load_data("data_bertopic.json")
data_all_desc = [item for item in data if isinstance(item.get("Description"), str)]
docs_all_fields = [clean_for_bertopic(combine_description_and_comments(ticket)) for ticket in data_all_desc]
topic_model_all_fields = BERTopic(min_topic_size=min_topic_size, embedding_model=embedding_model_name)
topics_all_fields, probs_all_fields = topic_model_all_fields.fit_transform(docs_all_fields)

save_bertopic_to_csv(
    model=topic_model_all_fields,
    docs=docs_all_fields,
    topics=topics_all_fields,
    filename=f"bertopic_summary_processeddata_min_topic_{min_topic_size}.csv"
)



Saved BERTopic summary to bertopic_summary_processeddata_min_topic_5.csv


In [ ]:
import pandas as pd

def generate_ticket_headings(topic_model, save_path="ticket_headings.csv"):
    # Get topic info
    topics_info = topic_model.get_topic_info()
    topic_names = topics_info['Name'].tolist()
    topic_ids = topics_info['Topic'].tolist()
    
    # Generate headings from topic names
    ticket_headings = [f"Ticket - {name.replace('_', ' ').capitalize()}" for name in topic_names]
    
    # Create DataFrame
    df = pd.DataFrame({
        "Topic_ID": topic_ids,
        "Ticket_Heading": ticket_headings
    })
    
    # Save as CSV
    df.to_csv(save_path, index=False)
    print(f"Ticket headings saved to: {save_path}")


Ticket headings saved to: ticket_headings.csv


In [12]:
topic_model_all_fields.visualize_topics()


In [13]:
topic_model_all_fields.visualize_hierarchy()


LDA

Create corpus for the LDA model

In [217]:
from gensim.models import TfidfModel
from gensim import corpora

def bigram_trigram_generator(tokenized_words, min_occurrence=5, threshold_score=100):
    bigram_phrases = gensim.models.Phrases(tokenized_words, min_count=min_occurrence, threshold=threshold_score)
    trigram_phrases = gensim.models.Phrases(bigram_phrases[tokenized_words], threshold=threshold_score)

    bigram = gensim.models.phrases.Phraser(bigram_phrases)
    trigram = gensim.models.phrases.Phraser(trigram_phrases)

    def make_bigrams(texts):
        return([bigram[doc] for doc in texts])

    def make_trigrams(texts):
        return ([trigram[bigram[doc]] for doc in texts])

    data_bigrams = make_bigrams(tokenized_words)
    data_bigrams_trigrams = make_trigrams(data_bigrams)
    return data_bigrams_trigrams

def corpus_generator(corpus_words, threshold_tfidf=0.04):
    texts = corpus_words

    # Step 1: Build initial dictionary and corpus
    id2word = corpora.Dictionary(texts)
    corpus = [id2word.doc2bow(text) for text in texts]

    # Step 2: Create TF-IDF model
    tfidf = TfidfModel(corpus, id2word=id2word)

    low_value = threshold_tfidf  # Try a much higher threshold

    # Step 3: Clean both texts and corpus
    filtered_texts = []
    filtered_corpus = []

    for i, bow in enumerate(corpus):
        tfidf_weights = tfidf[bow]
        tfidf_ids = {id for id, score in tfidf_weights if score >= low_value}
        bow_ids = {id for id, _ in bow}
        allowed_ids = tfidf_ids & bow_ids

        # Filtered BoW and Text
        new_bow = [b for b in bow if b[0] in allowed_ids]
        new_text = [id2word[b[0]] for b in new_bow]

        filtered_corpus.append(new_bow)
        filtered_texts.append(new_text)

    # Step 4: Rebuild dictionary and final corpus from cleaned text
    id2word = corpora.Dictionary(filtered_texts)
    corpus = [id2word.doc2bow(text) for text in filtered_texts]
    return filtered_texts, corpus, id2word

Create LDA for each category

In [ ]:
from collections import defaultdict
data = load_data("data_processed.json")
grouped_tickets = defaultdict(list)
for item in data:
    key = item.get("Custom field (Request Category)", "Unknown")
    grouped_tickets[key].append(item)
grouped_dict = dict(grouped_tickets)
grouped_ticket_by_request_category = [{"category": key, "items": value} for key, value in grouped_dict.items()]

descriptions_matrix_by_reqtype = []
for ticket_type in grouped_ticket_by_request_category:
    descriptions = [str(ticket["Description"]).split() for ticket in ticket_type["items"] if "Description" in ticket]
    descriptions_matrix_by_reqtype.append(descriptions)
words_matrix_by_reqtype = []
for descriptions_by_reqtype in descriptions_matrix_by_reqtype:
    words_matrix_by_reqtype.append(bigram_trigram_generator(descriptions_by_reqtype))
corpus_matrix_by_reqtype = []
for words_by_reqtype in words_matrix_by_reqtype:
    corpus_matrix_by_reqtype.append(corpus_generator(words_by_reqtype))
import gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import display, HTML

lda_models = []
lda_visualizations = []
lda_models_ = []
# Loop through each request type
for i, (words_by_reqtype, (filtered_texts, corpus, id2word)) in enumerate(zip(words_matrix_by_reqtype, corpus_matrix_by_reqtype)):
    
    # Skip if corpus is too small
    if len(corpus) < 10:
        print(f"Skipping request type index {i} (too few documents)")
        continue
    else:
        # Train LDA model
        lda_model = gensim.models.ldamodel.LdaModel(
            corpus=corpus,
            id2word=id2word,
            num_topics=8,
            random_state=100,
            update_every=1,
            chunksize=100,
            passes=10,
            alpha="auto",
            per_word_topics=True
        )
        lda_models_.append(lda_model)
        # Store model for reference
        lda_models.append((i, lda_model))

        # Create and display pyLDAvis visualization
        pyLDAvis.enable_notebook()
        vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
        lda_visualizations.append((i, vis))

        # Display with label
        request_type = grouped_ticket_by_request_category[i]['category']
        display(HTML(f"<h2 style='color: teal;'>Request Type: {request_type}</h2>"))
        display(vis)



Create a single LDA model for all descriptions

In [218]:
data = load_data("data_processed.json")
data_all_desc = [item for item in data if isinstance(item.get("Description"), str)]
tokenized_docs_lda = []
for doc in data_all_desc:
    tokenized_docs_lda.append(doc["Description"].split(" "))
bigrams_trigrams_LDA = bigram_trigram_generator(tokenized_docs_lda)
filtered_texts, corpus_lda, id2word_lda = corpus_generator(bigrams_trigrams_LDA, 0.2)
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_lda,
                                           id2word=id2word_lda,
                                           num_topics=17,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto",
                                           per_word_topics=True)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus_lda, id2word_lda, mds="mmds", R=30)
vis
pyLDAvis.save_html(vis, "lda_visualization_only_descriptions.html")


Create a single LDA model using descriptions and comments

In [221]:
data = load_data("data_processed.json")

# Step 1: Collect combined text (description + all comments)
combined_docs = []
for item in data:
    if isinstance(item.get("Description"), str):
        # Start with the description
        full_text = item["Description"]
        
        # Append available comment fields
        for i in range(20):  # From Comment to Comment.19
            comment_key = "Comment" if i == 0 else f"Comment.{i}"
            comment_text = item.get(comment_key)
            if isinstance(comment_text, str):
                full_text += " " + comment_text
        
        combined_docs.append(full_text)

# Step 2: Tokenize
tokenized_docs_lda = [doc.split(" ") for doc in combined_docs]

# Step 3: Generate bigrams/trigrams
bigrams_trigrams_LDA = bigram_trigram_generator(tokenized_docs_lda)

# Step 4: TF-IDF filtering and corpus preparation
filtered_texts, corpus_lda, id2word_lda = corpus_generator(bigrams_trigrams_LDA, threshold_tfidf=0.2)

# Step 5: Train LDA model
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus_lda,
    id2word=id2word_lda,
    num_topics=17,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha="auto",
    per_word_topics=True
)

# Step 6: Visualize
import pyLDAvis.gensim
import pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus_lda, id2word_lda, mds="mmds", R=30)
pyLDAvis.save_html(vis, "lda_visualization_with_comments.html")
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.339021  0.167333       1        1  13.558915
12     0.015194  0.389815       2        1  12.155312
13     0.310939  0.061649       3        1   8.804636
5     -0.090709  0.176703       4        1   7.420326
1      0.194280 -0.297886       5        1   7.410026
2      0.165880  0.210205       6        1   6.736403
4     -0.154708 -0.209054       7        1   6.522302
8     -0.190907  0.050021       8        1   5.943653
3      0.074608  0.105211       9        1   5.801801
15     0.164692 -0.108363      10        1   5.127972
6     -0.218729 -0.093232      11        1   4.892565
7      0.139313 -0.023754      12        1   4.824434
10    -0.011394 -0.263978      13        1   4.596550
9     -0.066346 -0.076949      14        1   2.541984
14     0.019232 -0.061711      15        1   1.937056
0     -0.015013 -0.006370      16        1   1.280877
16     0.002690 -0.019639      17        1   0.445186, topic_info=         Term       Freq       Total Category  logprob  loglift
41   business  123.00000  123.000000  Default  30.0000  30.0000
46       hour  118.00000  118.000000  Default  29.0000  29.0000
112   produce   84.00000   84.000000  Default  28.0000  28.0000
29     cancel   82.00000   82.000000  Default  27.0000  27.0000
22        add   58.00000   58.000000  Default  26.0000  26.0000
..        ...        ...         ...      ...      ...      ...
25   deadline    0.01357    2.697410  Topic17  -7.4518   0.1222
26     deduct    0.01357    1.430744  Topic17  -7.4518   0.7563
27     extend    0.01357    3.418618  Topic17  -7.4518  -0.1147
28      grant    0.01357   16.673650  Topic17  -7.4518  -1.6993
29     cancel    0.01357   82.680965  Topic17  -7.4518  -3.3004

[592 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
54       15  0.712853        aa_fe
327      14  0.788485  aa_fe_hello
172       4  0.627557        abide
173       4  0.627557      abiding
265       4  0.845804       access
...     ...       ...          ...
17       11  0.893961       window
242      14  0.480393        wrong
92       11  0.684870    yesterday
93       11  0.946037          yet
269       4  0.763835         zoom

[312 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 13, 14, 6, 2, 3, 5, 9, 4, 16, 7, 8, 11, 10, 15, 1, 17])